In [26]:
import torch
import SimpleITK
import numpy as np
import torch.nn.functional as F
from moviepy import *
from PIL import Image
from pathlib import Path
from itertools import islice
from skimage.transform import resize

In [27]:
data_crop_path = Path('/home/wangnannan/workdir/ultrasound/data/cropped_processed')
save_path = Path('/home/wangnannan/workdir/ultrasound/data/numpy_data')

abnormal_case_id_list = []
normal_case_id_list = []
for file in data_crop_path.iterdir():
    if file.name.startswith('ABN'):
        abnormal_case_id_list.append(file.name.split('#')[1])
    elif file.name.startswith('NOR'):
        normal_case_id_list.append(file.name.split('#')[1])

In [28]:
def load_video(video_path, sample_frame_num,height, width):
    clip = VideoFileClip(video_path)
    frame_list = []
    for frame_number, frame in enumerate(clip.iter_frames()):
        frame_array = np.array(frame)
        frame_list.append(frame_array)
    clip.close()

    video_data = np.stack(frame_list)
    video_data = torch.tensor(video_data,dtype=torch.float32).unsqueeze(dim=0)
    video_data = video_data.permute(0,4,1,2,3)
    video_data = F.interpolate(video_data, (16, 128, 128), mode="trilinear")[0].permute(1,2,3,0)
    video_data = video_data.numpy().astype(np.float32)
    return video_data

def load_user_data(case_list, sample_frame_num,height,width):
    user_data = np.zeros((4,sample_frame_num, height, width, 3),dtype=np.float32)
    
    for file in case_list:
        file_flag = file.name.split('#')[-1]
        video_data = load_video(file,sample_frame_num,height,width)
        
        if 'A' in file_flag:
            user_data[0] = video_data
        elif 'B' in file_flag:
            user_data[1] = video_data
        elif 'C' in file_flag:
            user_data[2] = video_data
        elif 'D' in file_flag:
            user_data[3] = video_data
            
    user_data = user_data.transpose(0,1,4,2,3)
    return user_data

In [30]:
for case_id in abnormal_case_id_list:
    case_list = list(data_crop_path.glob(f"*{case_id}*.*"))
    user_data = load_user_data(case_list, 16,128,128)
    flag = case_list[0].name.split('#')[0]
    np.save(save_path / f"{flag}_{case_id}.npy",user_data)

{'video_found': True, 'audio_found': False, 'metadata': {'software': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [490, 650], 'bitrate': 2179, 'fps': 15.33, 'codec_name': 'h264', 'profile': '(High)'}], 'input_number': 0}], 'duration': 3.07, 'bitrate': 2150, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [490, 650], 'video_bitrate': 2179, 'video_fps': 15.33, 'video_duration': 3.07, 'video_n_frames': 47}
/home/wangnannan/miniforge3/envs/torch12/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux-x86_64-v7.0.2 -i /home/wangnannan/workdir/ultrasound/data/cropped_processed/ABN#67160003#B1.avi -loglevel error -f image2pipe -vf scale=490:650 -sws_flags bicubic -pix_fmt rgb24 -vcodec rawvideo -
{'video_found': True, 'audio_found': False, 'metadata': {'software': 'Lavf61.7.100'}

In [32]:
for case_id in normal_case_id_list:
    case_list = list(data_crop_path.glob(f"*{case_id}*.*"))
    user_data = load_user_data(case_list, 16,128,128)
    np.save(save_path / f"NOR_{case_id}.npy",user_data)

{'video_found': True, 'audio_found': False, 'metadata': {'software': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [470, 630], 'bitrate': 2037, 'fps': 14.33, 'codec_name': 'h264', 'profile': '(High)'}], 'input_number': 0}], 'duration': 3.0, 'bitrate': 2007, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [470, 630], 'video_bitrate': 2037, 'video_fps': 14.33, 'video_duration': 3.0, 'video_n_frames': 42}
/home/wangnannan/miniforge3/envs/torch12/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux-x86_64-v7.0.2 -i /home/wangnannan/workdir/ultrasound/data/cropped_processed/NOR#66357895#A0.avi -loglevel error -f image2pipe -vf scale=470:630 -sws_flags bicubic -pix_fmt rgb24 -vcodec rawvideo -
{'video_found': True, 'audio_found': False, 'metadata': {'software': 'Lavf61.7.100'}, 

In [3]:
for i in Path('/home/wangnannan/workdir/ultrasound/data/numpy_data').iterdir():
    data = np.load(i)
    print(data.shape)

(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 256, 256)
(4, 32, 3, 25

# dog and cat

In [3]:
data_root = Path('/home/wangnannan/data/cat-dog_data/train')
cat_path = data_root / 'cat'
dog_path = data_root / 'dog'
save_path = Path('/home/wangnannan/workdir/5_3d_cls-2-master/data')

In [4]:
dog_list = list(dog_path.glob('*.jpg'))
sublists = [list(islice(dog_list, i, i+20)) for i in range(0, len(dog_list), 20)]

for i,sub_list in enumerate(sublists):
    array_list = []
    for file in sub_list:
        image = Image.open(file)
        image = image.resize((128,128))
        image = np.array(image)
        
        array_list.append(image)
    
    save_array = np.array(array_list).transpose(0,3,1,2)
    np.save(save_path / f'dog_{i}.npy',save_array)

In [5]:
dog_list = list(cat_path.glob('*.jpg'))
sublists = [list(islice(dog_list, i, i+20)) for i in range(0, len(dog_list), 20)]

for i,sub_list in enumerate(sublists):
    array_list = []
    for file in sub_list:
        image = Image.open(file)
        image = image.resize((128,128))
        image = np.array(image)
        
        array_list.append(image)
    save_array = np.array(array_list).transpose(0,3,1,2)
    np.save(save_path / f'cat_{i}.npy',save_array)